In [1]:
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.datasets import fetch_20newsgroups
import swifter
import re
import string
import pandas as pd

stop_words = list(set(stopwords.words('english')))

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[{}0-9]'.format(string.punctuation), ' ', text)
    text=re.sub(r'[^A-Za-z0-9 ]+', ' ', text)
    text = word_tokenize(text)
    text = [word for word in text if word not in stop_words]
    text = [WordNetLemmatizer().lemmatize(word) for word in text]
    text = ' '.join(text)
    return text

newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
df=pd.DataFrame({"content":newsgroups["data"]})

df["content"]=df["content"].swifter.apply(lambda x: preprocess_text(x))
df['content_length'] = df['content'].str.len()

df = df[df['content_length'] > 100]
df = df[df['content_length'] < 2000]

df=df[["content"]].reset_index(drop=True).reset_index().rename(columns={"index":"id"})
documents_20newsgroup=df.content.to_list()


Pandas Apply:   0%|          | 0/18846 [00:00<?, ?it/s]

In [2]:
l_chunck=20
sentences=[sentence.strip().lower() for sentence in documents_20newsgroup]
sentences  = ["".join([char for char in text if char not in string.punctuation]) for text in sentences]
sentences = [sentence.split(' ') for sentence in sentences]
sentences=[[token for token in sentence if len(token)>0] for sentence in sentences]
sentences=[sentence for sentence in sentences if len(sentence) > 5]
sentences = [sentence[i:i+l_chunck] for sentence in sentences for i in range(0, len(sentence), l_chunck) ]
sentences=[sentence for sentence in sentences if len(sentence) > 5]

In [3]:
print(len(sentences))

53363


In [4]:
import torch
import txt_to_pmi
DEVICE = torch.device('mps')
MODEL = txt_to_pmi.languagemodel.BERT(DEVICE, 'bert-base-cased', 256)
outs = txt_to_pmi.get_cpmi(MODEL, sentences, verbose=False)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
  0%|          | 32/53363 [02:04<57:43:59,  3.90s/it]


RuntimeError: MPS backend out of memory (MPS allocated: 14.51 GB, other allocations: 11.93 GB, max allowed: 27.20 GB). Tried to allocate 1.16 GB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).